In [1]:
!pip install cdsapi
import cdsapi
from datetime import date, datetime
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

  Created wheel for cdsapi: filename=cdsapi-0.5.1-py2.py3-none-any.whl size=11699 sha256=e5514798c576c7f1b712fa0d66cd096bf57e98dc4418448538514bb0a8f21240
  Stored in directory: /root/.cache/pip/wheels/b3/da/d4/76acb1b249ba411fb70ca7baf0d6ab7d6177e1f0d14f1383c1
Successfully built cdsapi


In [2]:
os.chdir("/root")
os.getcwd()

'/root'

In [3]:
!echo url: https://cds.climate.copernicus.eu/api/v2 >> .cdsapirc
!echo key: 90554:9d58d139-f8ac-4ed0-8ca2-81ffd5f162e5 >> .cdsapirc

In [19]:
#aller chercher la key ici : https://cds.climate.copernicus.eu/user/login
key = "9d58d139-f8ac-4ed0-8ca2-81ffd5f162e5"
variable = ['10m_u_component_of_wind', '10m_v_component_of_wind', 'total_precipitation',]
year = ['2018',]
#month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',]
month = ['10', '11',]
day =  ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15'
                    '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',]
time = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00','06:00','07:00','08:00','09:00','10:00','11:00','12:00',
                      '13:00','14:00','15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00',]
#area : [Nord, Ouest, Sud, Est]
area =  [-39.74, 51.90, -63.62, -50.75,]
nom_fichier = 'ml18_295b' 

In [20]:
def return_cdsapi(nom_fichier, key, variable, year, month, day, time, area):
  #aller chercher la key ici : https://cds.climate.copernicus.eu/user/login
  print('\n[------------]\n')
  os.chdir("/root")
  os.getcwd()

  key = key
  nom_fichier = nom_fichier + '.nc'

  c = cdsapi.Client()

  r = c.retrieve('reanalysis-era5-single-levels',
            {
              'product_type' : 'reanalysis',
              'variable' : variable,
              'year' : year,
              'month' : month,
              'day' : day,
              'time' : time,
              'area' : area,
              'format' : 'netcdf',
            },
            nom_fichier,
            )
  
  os.chdir("/content/csv")
  os.getcwd()
  r.download(nom_fichier)
  print('\n[===>--------]\n')

In [21]:
def mise_en_forme_nc(nom_fichier):
  my_example_nc_file = "/content/csv/" + nom_fichier + '.nc'
  fh = Dataset(my_example_nc_file, mode='r')

  lons = fh.variables['longitude'][:]
  lats = fh.variables['latitude'][:]
  u10 = fh.variables['u10'][:]
  v10 = fh.variables['v10'][:]
  tp = fh.variables['tp'][:]
  time_bis = fh.variables['time'][:]

  time_units = fh.variables['time'].units

  def transf_temps(time) :
    time_str = float(time)/24 + date.toordinal(date(1900,1,1))
    return date.fromordinal(int(time_str))

  time_ = [time_bis[i] for i in range(len(time_bis))]

  data = pd.DataFrame()
  data['time'] = time_
  data['time'] = data['time'].apply(lambda x : transf_temps(x))

  n = len(time)
  hour_ = time[0][0:2]
  data['hour'] = data['time']
  data['day'] = data['time']
  data['month'] = data['time']
  data['year'] = data['time']
  j=0

  for i in range(len(data)) :
      data["hour"].loc[i] = hour_
      if hour_ == time[len(time)-1][0:2]:
        j = 0
        hour_ = time[j][0:2]
      else : 
        j+=1
        hour_ = time[j][0:2]
      data['day'].loc[i] = data['time'].loc[i].day
      data['month'].loc[i] = data['time'].loc[i].month
      data['year'].loc[i] = data['time'].loc[i].year

  data['year'] = data['year'].apply(int)
  data['month'] = data['month'].apply(int)
  data['day'] = data['day'].apply(int)
  data['hour'] = data['hour'].apply(int)
  
  print('\n[======>-----]\n')

  return data, lons, lats, u10, v10, tp

In [22]:
def prepare_coord_target(nom_fichier):
  dir = "/content/csv/" + nom_fichier + '.csv'
  test_ = pd.read_csv(dir)
  test_.columns = ['time', 'long', 'lat', 'profondeur']
  test_['time'] = test_['time'].apply(lambda x : pd.Timestamp(datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')))

  test_['year'] = test_['time']
  test_['month'] = test_['time']
  test_['day'] = test_['time']
  test_['hour'] = test_['time']

  for i in range(len(test_)):
    test_['year'].loc[i] = test_['time'].loc[i].year
    test_['month'].loc[i] = test_['time'].loc[i].month
    test_['day'].loc[i] = test_['time'].loc[i].day
    test_['hour'].loc[i] = test_['time'].loc[i].hour

  group_gps = test_.groupby(by=['year','month', 'day', 'hour'])
  i=0
  moy_long = []
  moy_lat = []
  moy_prof = []


  for key,item in group_gps:
    a_group = group_gps.get_group(key)
    moy_long.append(np.mean(a_group['long']))
    moy_lat.append(np.mean(a_group['lat']))
    moy_prof.append(np.mean(a_group['prof']))

  test_fin = test_[['year', 'month', 'day', 'hour']].drop_duplicates()
  test_fin['long'] = moy_long
  test_fin['lat'] = moy_lat
  test_fin['year'] = test_fin['year'].apply(int)
  test_fin['month'] = test_fin['month'].apply(int)
  test_fin['day'] = test_fin['day'].apply(int)
  test_fin['hour'] = test_fin['hour'].apply(int)
  test_fin['profondeur'] = moy_prof

  print('\n[=========>--]\n')

  return test_fin.reset_index().drop(['index'], axis=1)

In [23]:
def prepare_coord_target2(nom_fichier):
  dir = "/content/csv/" + nom_fichier + '.csv'
  test_ = pd.read_csv(dir)

  group_gps = test_.groupby(by=['year','month', 'day', 'hour'])
  i=0
  moy_long = []
  moy_lat = []
  moy_prof = []


  for key,item in group_gps:
    a_group = group_gps.get_group(key)
    moy_long.append(np.mean(a_group['long']))
    moy_lat.append(np.mean(a_group['lat']))
    moy_prof.append(np.mean(a_group['prof']))

  test_fin = test_[['year', 'month', 'day', 'hour']].drop_duplicates()
  test_fin['long'] = moy_long
  test_fin['lat'] = moy_lat
  test_fin['year'] = test_fin['year'].apply(int)
  test_fin['month'] = test_fin['month'].apply(int)
  test_fin['day'] = test_fin['day'].apply(int)
  test_fin['hour'] = test_fin['hour'].apply(int)
  test_fin['profondeur'] = moy_prof

  print('\n[=========>--]\n')

  return test_fin.reset_index().drop(['index'], axis=1)

In [24]:
#df_spams_fin : df1, data : df2
def creation_complexe(df1, df2, u10, v10, tp, lats, lons):
  u10_ = []
  v10_ = []
  tp_ = []
  prof_ = []

  for i in range(len(df1)) :
    j=0
    n = len(df2)
    test_time = date(df2['year'].loc[j], df2['month'].loc[j], df2['day'].loc[j])
    test_hour = int(df2['hour'].loc[j])

    test_time_ = date(df1['year'].iloc[i], df1['month'].iloc[i], df1['day'].iloc[i])
    test_hour_ = int(df1['hour'].iloc[i])

    condition1 = test_time != test_time_
    condition2 = test_hour != test_hour_
    condition = condition1 or condition2

    while condition :
      if j == n-1 : 
        break
      test_time = date(df2['year'].loc[j], df2['month'].loc[j], df2['day'].loc[j])
      test_hour = int(df2['hour'].loc[j])
      condition1 = test_time != test_time_
      condition2 = test_hour != test_hour_
      condition = condition1 or condition2
      j+=1

    if j == n-1 : None

    else : 
      lat_test = df1['lat'].loc[i]
      long_test = df1['long'].loc[i]

      diff_lat = abs(lats-lat_test)
      idx_lat = np.argmin(diff_lat)

      diff_long = abs(lons-long_test)
      idx_long = np.argmin(diff_long)

      u10_.append(u10[j][idx_lat][idx_long])
      v10_.append(v10[j][idx_lat][idx_long])
      tp_.append(tp[j][idx_lat][idx_long])
      prof_.append(df1['profondeur'].loc[i])

  data_date = df2.merge(df1, on = ['year', 'month', 'day', 'hour'])
  data_fin = pd.DataFrame(data={'year':data_date['year'],
                               'month':data_date['month'],
                               'day':data_date['day'],
                               'hour':data_date['hour'],
                               'u10':u10_,
                               'v10':v10_,
                               'tp':tp_,
                                'profondeur':prof_})
  print('\n[============]\n')
  return data_fin

In [25]:
#type_crea == complexe si les coordonnées ne sont pas fixes
#df1 : mettre chemin du csv
def creation_finale(df1, nom_fichier, key, variable, year, month, day, time, area, type_crea = 'complexe', pret = True) :
  return_cdsapi(nom_fichier, key, variable, year, month, day, time, area)
  data, lons, lats, u10, v10, tp = mise_en_forme_nc(nom_fichier)
  if pret == True : 
    df = prepare_coord_target2(df1)
  else :  
    df = prepare_coord_target(df1)
  test = creation_complexe(df, data, u10, v10, tp, lats, lons)
  nom = nom_fichier + "_final.csv"
  test.to_csv(nom, index= False)
  return test

In [26]:
df = creation_finale("ml18_295b_coord_finale", nom_fichier, key, variable, year, month, day, time, area)


[------------]



2021-07-01 09:08:46,188 INFO Welcome to the CDS
2021-07-01 09:08:46,193 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-07-01 09:08:46,340 INFO Request is queued
2021-07-01 09:08:51,506 INFO Request is running
2021-07-01 09:15:06,637 INFO Request is completed
2021-07-01 09:15:06,639 INFO Downloading https://download-0005.copernicus-climate.eu/cache-compute-0005/cache/data8/adaptor.mars.internal-1625130755.7091937-12689-17-447ccb1e-fb18-4873-b8af-4c6d873e2004.nc to ml18_295b.nc (774M)
2021-07-01 09:17:17,941 INFO Download rate 5.9M/s
2021-07-01 09:17:17,944 INFO Downloading https://download-0005.copernicus-climate.eu/cache-compute-0005/cache/data8/adaptor.mars.internal-1625130755.7091937-12689-17-447ccb1e-fb18-4873-b8af-4c6d873e2004.nc to ml18_295b.nc (774M)
2021-07-01 09:20:39,022 INFO Download rate 3.8M/s



[===>--------]


[======>-----]


[=========>--]


[============]



In [27]:
df.head()

,year,month,day,hour,u10,v10,tp,profondeur
0,2018,10,22,21,7.820206,9.189895,0.000064,64.645879
1,2018,10,22,22,7.507843,9.230726,0.000031,48.085397
2,2018,10,22,23,7.336120,9.290860,0.000024,51.419499
3,2018,10,23,0,7.050176,9.489078,0.000033,32.760767
4,2018,10,23,1,6.683420,9.618253,0.000030,49.146080
